In [6]:
# Imports
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem, createBasicSubsystem, SimpleModel, SimpleReaction
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import libsbml
import bokeh.io
import bokeh.plotting

In [7]:
ss2 = createSubsystem ("SBML_Models/external_environment.xml")
ss3 = createSubsystem("SBML_Models/Full_Produce.xml")

# Membrane
# mb1 = createSubsystem("SBML_Models/rR1_Broc_membrane1_detailed.xml", membrane = True)
mb2 = createSubsystem("SBML_Models/rR1_P_membrane1_detailed.xml", membrane = True)
mb3 = createSubsystem("SBML_Models/simple_DFHBI_Prod.xml", membrane = True)

### Cell 3 ###
cell_3 =  System('cell_3',ListOfInternalSubsystems = [ss3],
                 ListOfExternalSubsystems = [ss2],
                 ListOfMembraneSubsystems = [mb2,mb3])
#cell_3 = System("cell_3")
#
#cell_3.setInternal([ss3])
#cell_3.setExternal([ss2])
#cell_3.setMembrane(mb2)
#cell_3.setMembrane(mb1)

cell_3_model = cell_3.getModel()

cell_3_model.renameCompartments('cell_3_external', 'actual_external')

##### Set Species Amount #####
#cell_P_model.setSpeciesAmount('rna_rR1_e', 0, compartment = 'actual_external')
cell_3_model.setSpeciesAmount('rna_rR1_e', 250, compartment = 'actual_external')
cell_3_model.setSpeciesAmount('DFHBI_e', 250, compartment = 'actual_external')

cell_3_model.setSpeciesAmount('Prod2_OFF', 50, compartment = 'cell_3_internal')
cell_3_model.setSpeciesAmount('dna_dAP', 50, compartment = 'cell_3_internal')
cell_3_model.setSpeciesAmount('protein_RNAseH', 0.1665, compartment = 'cell_3_internal') #0.1665
cell_3_model.setSpeciesAmount('protein_RNAP', 1.25, compartment = 'cell_3_internal')

cell_3_model.writeSBML('SBML_Models/cell_3_model.xml')

The subsystem from SBML_Models/rR1_P_membrane1_detailed.xml has multiple compartments
The subsystem from SBML_Models/simple_DFHBI_Prod.xml has multiple compartments


1

In [8]:
# Cell_P Identifiers #

P_id1 = cell_3_model.getSpeciesByName('complex_Prod2_ON').getId()
P_id2 = cell_3_model.getSpeciesByName('Prod2_OFF').getId()
P_id3 = cell_3_model.getSpeciesByName('DFHBI').getId()
P_id4 = cell_3_model.getSpeciesByName('BrocApt').getId()
P_id5 = cell_3_model.getSpeciesByName("complex_BrocApt_DFHBI").getId()
P_id6 = cell_3_model.getSpeciesByName("rna_rR1P").getId()
P_id7 = cell_3_model.getSpeciesByName("BrocApt_e", compartment="actual_external").getId()
P_id8 = cell_3_model.getSpeciesByName("rna_rR1_e", compartment="actual_external").getId()

P_id9 = cell_3_model.getSpeciesByName("complex_Prod2_AI").getId()

X_id2 = cell_3_model.getSpeciesByName("rna_rR1_e_sidP").getId()
X_id3 = cell_3_model.getSpeciesByName("DFHBI_e", compartment="actual_external").getId()

In [9]:
# Simulate with Bioscrape #
timepoints = np.linspace(0,28800,1000)
B,_ = cell_3_model.simulateWithBioscrape(timepoints)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(title="RNA External",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, B[P_id8], legend_label = "rna external" , color = "blue")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, B[P_id9], legend_label = "rR1:dA1" , color = "magenta")
#b.circle(timepoints, B[X_id2], legend_label = "Rna external:sid", color = "red")
b.legend.click_policy="hide"
b.legend.location="bottom_right"

c = bokeh.plotting.figure(title="RNA Internal",plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, B[P_id6], legend_label = "rna internal" , color = "purple")
c.legend.click_policy="hide"
c.legend.location="bottom_right"


bokeh.io.show(row(a, b, c))
warnings.filterwarnings("ignore")

Loading BokehJS ...

In [12]:
d = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
d.circle(timepoints, B[P_id7], legend_label = "Broc_Apt external" , color = "magenta")
#d.circle(timepoints, B[X_id3], legend_label = "DFHBI external" , color = "darkgreen")
d.legend.click_policy="hide"
d.legend.location="top_right"

e = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
e.circle(timepoints, B[P_id1], legend_label = "Prod On" , color = "blue")
e.circle(timepoints, B[P_id2], legend_label = "Prod Off" , color = "red")
e.legend.click_policy="hide"
e.legend.location="bottom_right"

bokeh.io.show(row(d, e))
warnings.filterwarnings("ignore")

In [13]:
f = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
f.circle(timepoints, B[P_id5], legend_label = "fluorescence" , color = "purple")
f.legend.click_policy="hide"
f.legend.location="bottom_right"

bokeh.io.show(row(f))
warnings.filterwarnings("ignore")